In [1]:
import pathlib
import pandas as pd

def read_data(folder: pathlib.Path):
    """Return dictionary of each cohort as a dataframe.

    Args:
        folder (pathlib.Path): Path to the folder containing the data.

    Returns:
        dict[str, pd.DataFrame]: Dictionary containing the dataframes for each cohort.
    """
    df = pd.read_csv(folder / "All_datasets_SomaScan_Plasma_7k_CVQC_SADRC_KADRC_ROSMAP_Kaci.csv")
    return {
        cohort: table for cohort, table in df.groupby("Cohort")
    }

def format_for_regression(df: pd.DataFrame, target_variable: str = "Age"):
    """Returns in X, y format for regression

    Args:
        df (pd.DataFrame): DataFrame with the data
        target_variable (str, optional): Target variable to use. Defaults to "Age".

    Returns:
        (pd.DataFrame, pd.DataFrame): X, y dataframes.
    """
    return df.iloc[:, 31:], df[target_variable]

In [2]:
folder = pathlib.Path("/Users/jameshaberberger/Gitlab/proteonn/data")
datasets = read_data(folder)

for k, output in datasets.items():
    print(k, output.shape[0])

/var/folders/2n/j06nrn2n7r524t776sngh0xr0000gr/T/ipykernel_95089/3169332940.py:13: DtypeWarning: Columns (2,6,16,17,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder / "All_datasets_SomaScan_Plasma_7k_CVQC_SADRC_KADRC_ROSMAP_Kaci.csv")


KADRC 3075
Kaci 188
ROSMAP 973
SADRC 1160


In [38]:
X, y = format_for_regression(datasets["KADRC"])

In [76]:
import GPy
import numpy as np
from tqdm import tqdm

class SparseUnivariateFeatureGPs:
    def __init__(self, X, y, num_inducing=10, kernel=None):
        """
        X: shape (n_samples, n_features)
        y: shape (n_samples,)
        Fits one sparse GP per feature using inducing points
        """
        if isinstance(X, np.ndarray) is False:
            X = X.detach().cpu().numpy()
        if isinstance(y, np.ndarray) is False:
            y = y.detach().cpu().numpy()

        self.X = X
        self.y = y.reshape(-1, 1)
        self.n_samples, self.n_features = X.shape
        self.num_inducing = num_inducing
        self.kernel = kernel or GPy.kern.RBF(input_dim=1)
        self.models = []

    def fit(self):
        self.models = []
        for i in tqdm(range(self.n_features)):
            xi = self.X[:, i:i+1]
            Z = np.linspace(xi.min(), xi.max(), self.num_inducing).reshape(-1, 1)
            m = GPy.models.SparseGPRegression(xi, self.y, kernel=self.kernel.copy(), Z=Z)
            m.optimize(messages=False)
            self.models.append(m)

    def predict(self, X_star):
        if isinstance(X_star, np.ndarray) is False:
            X_star = X_star.detach().cpu().numpy()

        preds = []
        variances = []

        for i, m in enumerate(self.models):
            xi_star = X_star[:, i:i+1]
            mu, var = m.predict(xi_star)
            preds.append(mu.ravel())
            variances.append(var.ravel())

        preds = np.stack(preds, axis=0)
        variances = np.stack(variances, axis=0)

        return preds, np.sqrt(variances)


In [78]:
mugp = SparseUnivariateFeatureGPs(
    X.values,
    y.values,
    num_inducing=10,
)
mugp.fit()

  0%|          | 2/7040 [00:00<24:45,  4.74it/s] /opt/homebrew/lib/python3.11/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /opt/homebrew/lib/python3.11/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /opt/homebrew/lib/python3.11/site-packages/GPy/kern/src/rbf.py:178: RuntimeWarning:invalid value encountered in multiply
  1%|          | 60/7040 [00:12<23:17,  4.99it/s]


KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: 

In [86]:
import torch
import gpytorch

class SparseUnivariateFeatureGPs:
    def __init__(self, X, y, num_inducing=10, device="cpu"):
        """
        X: (n_samples, n_features), y: (n_samples,)
        """
        torch.set_default_dtype(torch.float32)

        self.device = torch.device(device if torch.backends.mps.is_available() else "cpu")
        self.X = torch.tensor(X, dtype=torch.float32, device=self.device)
        self.y = torch.tensor(y, dtype=torch.float32, device=self.device)
        self.n_samples, self.n_features = self.X.shape
        self.num_inducing = num_inducing
        self.models = []

    class FeatureGPModel(gpytorch.models.ApproximateGP):
        def __init__(self, inducing_points):
            variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
                inducing_points.size(0), dtype=torch.float32
            )
            variational_strategy = gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            )
            super().__init__(variational_strategy)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.RBFKernel()
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    def fit(self, num_steps=200, lr=0.1):
        self.models = []
        self.likelihoods = []

        for i in tqdm(range(self.n_features)):
            xi = self.X[:, i].unsqueeze(-1)
            inducing = xi[torch.randperm(len(xi))[:self.num_inducing]]

            model = self.FeatureGPModel(inducing.float()).to(self.device)
            likelihood = gpytorch.likelihoods.GaussianLikelihood().to(self.device)

            model.train()
            likelihood.train()

            optimizer = torch.optim.Adam([
                {'params': model.parameters()},
                {'params': likelihood.parameters()},
            ], lr=lr)

            mll = gpytorch.mlls.VariationalELBO(likelihood, model, self.y.numel())

            for _ in range(num_steps):
                optimizer.zero_grad()
                output = model(xi.float())
                loss = -mll(output, self.y)
                loss.backward()
                optimizer.step()

            self.models.append(model.eval())
            self.likelihoods.append(likelihood.eval())

    def predict(self, X_star):
        X_star = torch.tensor(X_star, dtype=torch.float32, device=self.device)
        preds = []
        stds = []

        for i, (model, likelihood) in enumerate(zip(self.models, self.likelihoods)):
            xi_star = X_star[:, i].unsqueeze(-1)
            with torch.no_grad(), gpytorch.settings.fast_pred_var():
                pred = likelihood(model(xi_star))
                preds.append(pred.mean.cpu())
                stds.append(pred.stddev.cpu())

        return torch.stack(preds), torch.stack(stds)


In [ ]:
mugp = SparseUnivariateFeatureGPs(
    X.values,
    y.values,
    num_inducing=10,
)
mugp.fit()

  0%|          | 0/7040 [00:00<?, ?it/s]

  0%|          | 30/7040 [00:23<1:31:32,  1.28it/s]